In [22]:
# Define the folder path where the files are stored
input_path = "../data/test/raw/papers/"  # Replace with the actual folder path
output_path = "../data/test/interim/teis/"  # Replace with the desired output folder path

#input_path = "../data/raw/downloaded_papers/from-scopus"  # Replace with the actual folder path
#output_path = "../data/interim/teis/from-scopus"  # Replace with the desired output folder path

In [ ]:
grobid_server = "https://kermitt2-grobid.hf.space"

In [23]:
# Write python code that reads all filenames from folder FOLDER and for each filename runs the command
# curl -v -H "Accept: application/xml" --form consolidateCitations=1 --form includeRawCitations=1 --form segmentSentences=1 --form input=@./FILENAME localhost:8070/api/processFulltextDocument > FILENAME + "tei.xml"

In [24]:
import requests
import time

def check_grobid_server():
    # https://grobid.readthedocs.io/en/latest/Grobid-service/
    url = "http://localhost:8070/api/isalive"
    
    try:
        # Send a GET request to the URL
        # 
        print(f"checking {url}...")
        response = requests.get(url)
        print("checked!")

        # Check if the status code indicates success (200-299)
        if response.status_code == 200 and response.text == "true":
            print(f"Server is up and running at {url}.")
            return True
        else:
            print(f"Server responded with status code: {response.status_code}")
            return False
    
    except requests.ConnectionError:
        print(f"Unable to connect to {url}. The server may be down or unreachable.")
        return False
    
# Test the server
check_grobid_server()

checking http://localhost:8070/api/isalive...
Unable to connect to http://localhost:8070/api/isalive. The server may be down or unreachable.


False

In [25]:
'''
import docker
import time

def bring_grobid_server():

    if check_grobid_server():
        return

    # Initialize Docker client
    client = docker.from_env()

    # Define the container configuration
    container_name = "grobid"
    image_name = "grobid/grobid:0.8.1"
    host_port = 8070
    container_port = 8070
    volume_mapping = {
        r"C:/Users/pedro/Downloads/AUSTRIAN/CONTEXT_RESEARCH/grobid.yaml": {
            "bind": "/opt/grobid/grobid-home/config/grobid.yaml",
            "mode": "ro"
        }
    }

    try:
        # Run the container
        print("Starting the Docker container...")
        container = client.containers.run(
            image=image_name,
            init=True,
            remove=True,
            ulimits=[docker.types.Ulimit(name="core", soft=0, hard=0)],
            ports={f"{container_port}/tcp": host_port},
            volumes=volume_mapping,
            detach=True  # Run container in detached mode
        )
        time.sleep(90)
        print(f"Container started. ID: {container.id}")
        print(f"Visit http://localhost:{host_port} to access the service.")
    except Exception as e:
        print(f"Error: {e}")


bring_grobid_server()
'''

'\nimport docker\nimport time\n\ndef bring_grobid_server():\n\n    if check_grobid_server():\n        return\n\n    # Initialize Docker client\n    client = docker.from_env()\n\n    # Define the container configuration\n    container_name = "grobid"\n    image_name = "grobid/grobid:0.8.1"\n    host_port = 8070\n    container_port = 8070\n    volume_mapping = {\n        r"C:/Users/pedro/Downloads/AUSTRIAN/CONTEXT_RESEARCH/grobid.yaml": {\n            "bind": "/opt/grobid/grobid-home/config/grobid.yaml",\n            "mode": "ro"\n        }\n    }\n\n    try:\n        # Run the container\n        print("Starting the Docker container...")\n        container = client.containers.run(\n            image=image_name,\n            init=True,\n            remove=True,\n            ulimits=[docker.types.Ulimit(name="core", soft=0, hard=0)],\n            ports={f"{container_port}/tcp": host_port},\n            volumes=volume_mapping,\n            detach=True  # Run container in detached mode\n    

In [26]:
#assert check_grobid_server()

In [27]:
import os

def truncate_filename(file_path):
    """
    Truncate the filename if it exceeds 100 characters and rename the file.

    Args:
        file_path (str): The full file path.

    Returns:
        str: The new file path with the truncated filename if necessary.
    """
    # Extract the directory, filename, and extension
    directory, file_name = os.path.split(file_path)
    name, ext = os.path.splitext(file_name)

    print('len', len(file_name))

    # Check if the filename (excluding the directory) is greater than 100 characters
    if len(file_name) > 110:
        # Truncate the filename to ensure it doesn't exceed 100 characters
        trunc_length = 110 - len(ext)
        name = name[:trunc_length]
        file_name = name + ext

        # Create the new file path
        new_file_path = os.path.join(directory, file_name)

        print('truncating filename...')
        # Rename the file
        try:
             os.rename(file_path, new_file_path)
        except FileExistsError:
            print(f"Error: The directory '{directory}' already exists.")
            return new_file_path

        return new_file_path

    # If no truncation is needed, return the original file path
    return file_path

# Example usage
file_path = "/some/long/path/to/a/file/with_a_really_long_filename_that_needs_to_be_truncated_because_it_is_too_long.txt"
new_file_path = truncate_filename(file_path)
print("Original:", file_path)
print("Modified:", new_file_path)


len 81
Original: /some/long/path/to/a/file/with_a_really_long_filename_that_needs_to_be_truncated_because_it_is_too_long.txt
Modified: /some/long/path/to/a/file/with_a_really_long_filename_that_needs_to_be_truncated_because_it_is_too_long.txt


In [28]:
import os
import subprocess
import time
import random


# Make sure the output folder exists
if not os.path.exists(output_path):
    os.makedirs(output_path)

# List all files in the folder
files = []

for filename in os.listdir(input_path):
    file_path = os.path.join(input_path, filename)
    if os.path.isfile(file_path):
        # Add the file and its size to the list
        files.append((file_path, os.path.getsize(file_path)))

# Sort files by size (smallest first)
files.sort(key=lambda x: x[1])
random.shuffle(files)

# Process each file in order of size
for file_path, file_size in files:
    # Skip processing if the file size is zero
    #if file_size == 0:
    #    print(f"Skipped {file_path} (file size is zero)")
    #    continue

    # Extract the filename from the path
    filename = os.path.basename(file_path)

    # Prepare the output filename
    output_filename = f"{filename}.grobid.tei.xml"
    output_file_path = os.path.join(output_path, output_filename)

    # Check if the output file exists and its size is greater than 0
    if not os.path.exists(output_file_path) or os.path.getsize(output_file_path) == 0:
        # Prepare the curl command
        print(f"Let's try {file_path}")
        #time.sleep(10)
        #if not check_grobid_server():
        #    bring_grobid_server()

        file_path = truncate_filename(file_path)
            
        curl_command = [
            "curl", "-v", 
            "-H", "Accept: application/xml", 
            "--form", "consolidateCitations=1", 
            "--form", "includeRawCitations=1", 
            "--form", "segmentSentences=1",
            "--form", "generateIDs=1", 
            "--form", f"input=@{file_path}", 
            "https://kermitt2-grobid.hf.space/api/processFulltextDocument",
            #"localhost:8070/api/processFulltextDocument"
        ]

        # Open the output file to redirect the curl output into it
        with open(output_file_path, "w") as output_file:
            # Run the curl command and capture the output
            print(file_path)
            try:
                #cmd = f'curl -v -H "Accept: application/xml" --form "consolidateCitations=1" --form "includeRawCitations=1" --form "segmentSentences=1" --form "generateIDs=1" --form "input=@{file_path}" localhost:8070/api/processFulltextDocument'
                #print(cmd)
                # Run the process with a timeout of 5 minutes (300 seconds)
                result = subprocess.run(curl_command, stdout=output_file, stderr=subprocess.PIPE, timeout=600)
                if result.stderr:
                        print("STDERR:")
                        print(result.stderr.decode())  # Decode from bytes to string

            except subprocess.TimeoutExpired:
                print("The process did not finish within 10 minutes and was terminated.")

        # Check if the output file size is zero
        if os.path.getsize(output_file_path) == 0:
            print(f"{output_file_path} is empty. NOT PROCESSED.")
            #break
        else:
            print(f"Processed {filename} -> {output_file_path}")
            time.sleep(60)
    else:
        print(f"Skipped {filename} (output file already exists and is non-empty)")


Let's try ../data/test/raw/papers/Demonstrated-risk-preferences-and-COVID19-regulations-in-the-United-States_2024_Springer.pdf
len 92
../data/test/raw/papers/Demonstrated-risk-preferences-and-COVID19-regulations-in-the-United-States_2024_Springer.pdf
STDERR:
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0* Host kermitt2-grobid.hf.space:443 was resolved.
* IPv6: (none)
* IPv4: 3.214.53.138, 3.232.180.39, 54.208.116.228
*   Trying 3.214.53.138:443...
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0* schannel: disabled automatic use of client certificate
* ALPN: curl offers http/1.1
* ALPN: server accepted http/1.1
* Connected to kermitt2-grobid.hf.space (3.214.53.138) port 443
* using HTTP/1.x
> POST /api/processFulltextDocument HTTP/1.1
> Host: kermitt2-grobid.hf.space
> 

KeyboardInterrupt: 